## Import

In [1]:
import re
import json
import jieba
import string
import numpy as np
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from zhon.hanzi import punctuation
from multiprocessing.pool import ThreadPool as Pool

In [2]:
with open('../../data/THUOCL_medical.txt', 'r', encoding='utf-8') as f:
    list_word_freq = f.readlines()
list_word_freq = [ i.strip().split() for i in list_word_freq]
list_word_freq = [ [word, int(freq)] for word, freq in list_word_freq]
for (word, freq) in list_word_freq:
    jieba.add_word(word.strip(),tag=freq)
    jieba.suggest_freq(word, tune=True)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\hp\AppData\Local\Temp\jieba.cache
Loading model cost 0.611 seconds.
Prefix dict has been built successfully.


In [2]:
path_file_data = '../../data/MedQA/Mainland/test.jsonl'

## Data

In [3]:
list_dict_test = []
with open(path_file_data, 'r', encoding="utf-8") as f:
    for idx, line in enumerate(f):
        data = json.loads(line)
        data['ID'] = idx
        data['A'], data['B'], data['C'], data['D'], data['E'] = data['options']['A'], data['options']['B'], data['options']['C'], data['options']['D'], data['options']['E']
        del data['options']
        list_dict_test.append(data)

In [4]:
list_dict_test[0]

{'question': '经调查证实出现医院感染流行时，医院应报告当地卫生行政部门的时间是（\u3000\u3000）。',
 'answer': '24小时内',
 'meta_info': '卫生法规',
 'answer_idx': 'E',
 'ID': 0,
 'A': '2小时',
 'B': '4小时内',
 'C': '8小时内',
 'D': '12小时内',
 'E': '24小时内'}

## Corpus

In [9]:
with open("../../data/textbook/zh_paragraph/all_books.txt", 'r', encoding='utf-8') as f:
    list_corpus_textbooks = f.readlines()

In [10]:
def clean_string(str_processing, flag_en=False, flag_en_punc=False, flag_cn_punc=False, flag_num=False):
    # remove English character
    if flag_en:
        str_processing = re.sub('[a-zA-Z]','',str_processing)
    # remove English punctuation
    if flag_en_punc:
        str_processing = re.sub('[{}]'.format(string.punctuation),"",str_processing)
    # remove Chinese punctuation
    if flag_cn_punc:
        str_processing = re.sub('[{}]'.format(punctuation),"",str_processing) 
    # remove Numeric char
    if flag_num:
        str_processing = re.sub('[\d]','',str_processing) # [0-9]
    return str_processing

In [11]:
def chinese_in(word):
    for ch in str(word):
        if '\u4e00' <= ch <= '\u9fff':
            return True
    return False

In [12]:
def flatten(lst):
    result = []
    for item in lst:
        if isinstance(item, list):
            result.extend(flatten(item))
        else:
            result.append(item)
    return result

In [13]:
def remove_duplicates(lst):
    seen = {}
    result = []
    for item in lst:
        if item not in seen:
            seen[item] = True
            result.append(item)
    return result

In [14]:
def loading_knowledge(path_file):
    with open(path_file, 'r', encoding="utf-8") as f:
        data_new = json.load(f)
    list_knowledge = [ dict_one['Q+A']+dict_one['Q+B']+dict_one['Q+C']+dict_one['Q+D']+dict_one['Q+E'] for dict_one in data_new ]
    list_knowledge = remove_duplicates(flatten(list_knowledge))
    return list_knowledge

In [16]:
list_corpus = remove_duplicates(list_corpus_textbooks)
print(f'Loading {len(list_corpus)} knowledge')

Loading 110030 knowledge


In [17]:
list_corpus_fined = []
for line_one in list_corpus:
    line_one_cleaned = clean_string(line_one, flag_en=True, flag_en_punc=True, flag_cn_punc=True, flag_num=True)
    if len(line_one_cleaned)>10 and chinese_in(line_one_cleaned):
        list_corpus_fined.append(line_one.strip())
# list_corpus_fined = [ line_one.strip() for line_one in list_corpus_fined if len(line_one)>10 and chinese_in(line_one) ]
print(f'Filtered knowledge: {len(list_corpus_fined)} items')

Filtered knowledge: 86785 items


## BM

In [18]:
list_corpus_fined[666]

'处方药向非处方药的转换，受国家和地区医药卫生水平、经济状况及民众文化素质诸因素的影响。事实上，不同国家和地区非处方药的种类存在差异。一种药物，在某一国限定为处方药，而在另一国，则可能已转换成非处方药，这显然与当地医药卫生状况及民众文化素质有关。当然，某种药品在其他国作为非处方药应用的成功经验，亦可以作为本国申请或批准同一药品为非处方药的借鉴信息。'

In [19]:
corpus_knowledge = [ list(jieba.cut(doc)) for doc in list_corpus_fined]
bm25_knowledge = BM25Okapi(corpus_knowledge)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\hp\AppData\Local\Temp\jieba.cache
Loading model cost 0.515 seconds.
Prefix dict has been built successfully.


## Func of find

In [20]:
def find_knowledge_question(bm, bm_corpus, question, top_n, flag_print=None):
    tokenized_query = list(jieba.cut(question))
    if flag_print:    
        print(tokenized_query)
    list_similar = bm.get_top_n(tokenized_query, bm_corpus, n=top_n)
    list_similar = [ ''.join(list_str) for list_str in list_similar ]
    return list_similar

In [21]:
def find_knowledge_option(bm, bm_corpus, option, top_n, flag_print=None):
    tokenized_query = list(jieba.cut(option))
    if flag_print:    
        print(tokenized_query)
    list_similar = bm.get_top_n(tokenized_query, bm_corpus, n=top_n)
    list_similar = [ ''.join(list_str) for list_str in list_similar ]
    return list_similar

In [24]:
# find knowledge for each question+option
def find_knowledge_question_option(bm, bm_corpus, question, option, top_n, flag_print=None):
    # Only question: if only number after removing punctuation
    if clean_string(option, flag_en_punc=True, flag_cn_punc=True).isnumeric():
        query = question
    # repeat option to same length of question
    else:
        query = question + ' '  +  option
    tokenized_query = list(jieba.cut(query))
    if flag_print:
        print(tokenized_query)
    list_similar = bm.get_top_n(tokenized_query, bm_corpus, n=top_n)
    list_similar = [ ''.join(list_str) for list_str in list_similar ]
    return list_similar

In [30]:
idx = 78
question = list_dict_test[idx]['question']
option = list_dict_test[idx]['A']
print(question, option)

营养不良最先出现的症状是（　　）。 体重不增


In [31]:
list_knowledge_question = find_knowledge_question(bm25_knowledge, corpus_knowledge, question, top_n=5, flag_print=True)
list_knowledge_question

['营养不良', '最先', '出现', '的', '症状', '是', '（', '\u3000', '\u3000', '）', '。']


['广义的营养不良(malnutrition)包括营养低下(undernutrition)和营养过度(overnutrition)两方面，本节阐述为前者，即由千各种原因引起的蛋白质和（或）热能摄入不足或消耗增多引起的营养缺乏病，又称蛋白质热能营养不良(protein-energy malnut rition, PEM)多见千3岁以下婴幼儿。根据临床表现，可分为消瘦型(marasmu s)（由于热能严重不足引起），水肿型(kwashiorkor)（由千严重蛋白质缺乏引起）和混合型（又称消瘦－水肿型，临床表现介于两者之间）。我国儿童以消瘦型营养不良多见，混合型营养不良次之，水肿型营养不良较为罕见。目前儿童营养不良在全球范围内仍是威胁儿童生长健康的一个重要疾病，在许多第三世界国家，营养不良仍是儿童死亡的主要原因，约占儿童死亡起因的1/3。流行病学调查显示，目前我国严重营养不良已经很少见，多继发千某些慢性疾病。但因为喂养不当和（或）小儿饮食习惯不良，如偏食、挑食等，造成轻至中度的营养不良发病率仍较高，且轻症及早期营养不良的症状和体征不典型，易漏诊，必须通过详细询问病史、细致的体格检查以及结合实验室检查进行诊断。一旦出现营养不良，如果不能及时纠正，尤其在小婴儿，可严重影响患儿的生长、智力发育及免疫功能，易患各种感染性疾病，应引起足够重视。',
 '腹泻病时还可合并低钙血症和低镁血症：腹泻患儿进食少，吸收不良，从大便丢失钙、镁，可使体内钙、镁减少，此症在活动性侚倓病和营养不良患儿更多见。但是脱水、酸中毒时由于血液浓缩、离子钙增多等原因，不出现低钙的症状，待脱水、酸中毒纠正后则出现低钙症状（手足描据和惊厥）。极少数久泻和营养不良患儿输液后出现震颤、抽搞。用钙治疗无效时应考虑有低镁血症的可能。',
 '根据小儿年龄及喂养史、体重下降、皮下脂肪减少、全身各系统功能紊乱及其他营养素缺乏的临床症状和体征，典型病例的诊断并不困难。诊断营养不良的基本测量指标为身高（长）和体重。5岁以下儿童营养不良的分型和分度如下。',
 '［临床表现］短肠综合征病人早期最主要的临床表现为腹泻、水和电解质失衡，以及营养不良，其中腹泻一般最早出现，其严重程度与残留肠管的长度密切相关。腹泻导致进行性脱水、血容量降低，水、电解质紊乱和酸碱失衡。后期腹泻渐趋减少，根据残留肠管的长度与代偿情况，病人的营养状况可得到维持

In [32]:
list_knowledge_option = find_knowledge_option(bm25_knowledge, corpus_knowledge, option, top_n=5, flag_print=True)
list_knowledge_option

['体重', '不增']


['(2)营养不良：因呕吐及食管炎引起喂食困难而营养摄取不足所致。主要表现为体重不增和生长发育迟缓、贫血。',
 '3.婴儿暂时性糖尿病因不明，可能与患儿胰岛B细胞功能发育不够成熟有关。多在出生后6周内发病，表现为发热、呕吐、体重不增、脱水等症状。血糖增高，尿糖及酮体阳性，经补液等一般处理或给予小量胰岛素即可恢复。对这类患儿应进行葡萄糖耐量试验和长期随访，以与1型糖尿病鉴别。',
 '新生儿不会用语言交流，常用哭啼表现身心变化和需求。需要爱抚时哭声清脆响亮；饥饿时哭声很大，直至不适解除；生病时会长时间哭闹，但当疾病严重时，哭声低甚至不哭，并伴不吃、不动、体重不增等。医务人员熟练掌握观察病情的技巧，操作时动作轻巧、敏捷，并应该用语言和抚触等给予无微不至的关爱和呵护（图9-3)。',
 '2.高钙血症见于维生素D中毒、甲状旁腺功能亢进症等。3低钾血症见于原发性醒固酮增多症、慢性腹泻、B釭tter综合征等。4继发性肾性多尿见千慢性肾炎、慢性肾盂肾炎等导致慢性肾功能减退时。5原发性肾性尿崩症为X连锁或常染色体显性遗传疾病，是由于肾小管上皮细胞对AVP无反应所致。发病年龄和症状轻重差异较大，重者生后不久即出现症状，可有多尿、脱水、体重不增、生长障碍、发热、末梢循环衰竭甚至中枢神经系统症状。轻者发病较晚，当患儿禁饮时，可出现高热、末梢循环衰竭、体重迅速下降等症状。禁水、加压素试验均不能提高尿渗透压。',
 '早产儿肾浓缩功能更差，肾小管对睦固酮反应低下，对钠的重吸收功能差，易出现低钠血症。葡萄糖阙值低，易发生糖尿。碳酸氢根阙值极低和肾小管排酸能力差。由于普通牛乳中蛋白质含量及酪蛋白比例均高，可致内源性氢离子增加，当超过肾小管的排泄能力时，引起晚期代谢性酸中毒(late metabolic acidosis)，表现为面色苍白、反应差、体重不增和代谢性酸中毒。因此人工喂养的早产儿应采用早产儿配方奶粉。']

In [33]:
list_knowledge_question_option = find_knowledge_question_option(bm25_knowledge, corpus_knowledge, question, option, top_n=5, flag_print=True)
list_knowledge_question_option

['营养不良', '最先', '出现', '的', '症状', '是', '（', '\u3000', '\u3000', '）', '。', ' ', '体重', '不增']


['(2)营养不良：因呕吐及食管炎引起喂食困难而营养摄取不足所致。主要表现为体重不增和生长发育迟缓、贫血。',
 '营养不良的早期表现是活动减少，精神较差，体重生长速度不增。随营养不良加重，体重逐渐下降，主要表现为消瘦。皮下脂肪层厚度是判断营养不良程度重要指标之一。皮下脂肪消耗的顺序先是腹部，其次为躯干、臀部、四肢，最后为面颊。皮下脂肪逐渐减少以致消失，皮肤干燥、苍白、渐失去弹性，额部出现皱纹，肌张力渐降低、肌肉松弛、肌肉萎缩呈“皮包骨”时，四肢可有挛缩。营养不良初期，身高不受影响，但随病情加重，骨骼生长减慢，身高亦低于正常。轻度PEM精神状态正常；重度可有精神委靡，反应差，体温偏低，脉细无力，无食欲，腹泻、便秘交替。血浆白蛋白明显下降时出现凹陷性水肿国，严重时感染形成慢性溃疡。重度营养不良可伴有重要脏器功能损害。',
 '根据小儿年龄及喂养史、体重下降、皮下脂肪减少、全身各系统功能紊乱及其他营养素缺乏的临床症状和体征，典型病例的诊断并不困难。诊断营养不良的基本测量指标为身高（长）和体重。5岁以下儿童营养不良的分型和分度如下。',
 '神经性厌食患者由千体重下降出现严重的营养不良，所以首先要纠正营养不良以及由千营养不良所带来的水电解质平衡紊乱，给予足够维持生命的能最，以挽救患者生命。需要制订合理的饮食计划，通过增加饮食、加强营养，逐渐恢复正常体重和身体健康。',
 '由于患者限制饮食，体重下降明显，常常会出现营养不良和代谢紊乱，如皮肤干燥、苍白、皮下脂肪少、失去弹性与光泽，毛发稀疏脱落，低血压，低体温，心动过缓，贫血，水肿及无症状性低血糖等。呕吐和滥用泻药可能导致各种电解质紊乱如血脂、水电解质和酸碱平衡紊乱等症状，最严重的是低钾血症。化验检查可见白细胞减少和肝肾功能改变。随着疾病的发展，会出现越来越严重的营养不良、消瘦、疲劳和肌肉无力，严重者可发展为恶病质，甚至死亡。当体重低于正常体重的60％时，死亡率较高。']

## Data

In [34]:
def get_knowledge(set_added, list_bank):
    idx = 0
    while list_bank[idx] in set_added:
        idx += 1
        if idx==len(list_bank):
            return None
    set_added = set_added.add(list_bank[idx])
    return list_bank[idx]

In [35]:
# Run
def get_kg_question_option(dict_test):
    question = dict_test['question']
    # set_knowledge = set()
    for option in ['A', 'B', 'C', 'D', 'E']:
        option_name = dict_test[option]
        list_knowledge_question_option = find_knowledge_question_option(bm25_knowledge, corpus_knowledge, question, option_name, top_n=5)
        dict_test[f"Q+{option}"] = list_knowledge_question_option
        # dict_test[f"Q+{option}"] = get_knowledge(set_knowledge, list_knowledge_question_option)
    return dict_test

In [ ]:
for dict_test in tqdm(list_dict_test):
    dict_test = get_kg_question_option(dict_test)

In [23]:
pool = Pool(10)
list_dict_test_new = pool.map(get_kg_question_option, tqdm(list_dict_test[:50]))
pool.close()
pool.join()

100%|██████████| 50/50 [00:00<00:00, 24989.90it/s]


In [87]:
with open('../../data/data_knowledge_enhancement.json', "w", encoding="utf-8") as f:
    f.write(json.dumps(list_dict_test_new, indent=2, ensure_ascii=False))